# How to Quickly Start Competing in the Kaggle Data Science Bowl 2017 with CNTK and LightGBM

In this notebook, we share a baseline script that allows anyone interested in participating in the [Data Science Bowl Lung Cancer Detection Competition](https://www.kaggle.com/c/data-science-bowl-2017) to create the first submission quickly. We first extract features from the CT scan images of lungs using a pretrained convolutional neural network (CNN) implemented on [CNTK](https://github.com/Microsoft/CNTK/wiki), followed by running a boosted tree classifier implemented on [LightGBM](https://github.com/Microsoft/LightGBM).  Both CNTK and LightGBM have been shown to have excellent training speed in a range of tasks (see, for examples, [this paper](https://arxiv.org/abs/1608.07249) for CNTK and [this summary](https://github.com/Microsoft/LightGBM/wiki/Experiments#comparison-experiment) for LightGBM), and this should help participants iterate on their models quickly.  Furthermore, we use [GPU acceleration available on Azure Data Science Virtual Machines](http://aka.ms/dsvm/deeplearning) to speed up computations in CNN featurization.  

With these three tools, one can achieve a **score of 0.55979** on the leaderboard within approximately **one hour** (excluding data download and virtual machine startup time). 


This work is also published as a [blog in Technet](https://blogs.technet.microsoft.com/machinelearning/2017/02/17/quick-start-guide-to-the-data-science-bowl-lung-cancer-detection-challenge-using-deep-learning-microsoft-cognitive-toolkit-and-azure-gpu-vms/), as a [notebook in Cortana Gallery](https://gallery.cortanaintelligence.com/Notebook/Medical-Image-Recognition-for-the-Kaggle-Data-Science-Bowl-2017-with-CNTK-and-LightGBM-1) and as a [script in Kaggle](https://www.kaggle.com/hoaphumanoid/data-science-bowl-2017/cntk-and-lightgbm-quick-start/).  


## Dependencies

The Azure Data Science Virtual Machine (DSVM) comes with CNTK, LightGBM, OpenCV, Scikit-learn preinstalled, so you don't have to worry about the configuration. 

Apart from the libraries, we are going to use a pretrained convolutional neural network (CNN) with the [ResNet architecture](https://arxiv.org/abs/1512.03385). This configuration, developed by Microsoft Research, was the first CNN to surpass the human level performance in image classification. The network was trained in the [ImageNet dataset](http://image-net.org/) and can be downloaded [here](https://migonzastorage.blob.core.windows.net/deep-learning/models/cntk/imagenet/ResNet_152.model).


## Data
In addition to the libraries you have to download the [data](https://www.kaggle.com/c/data-science-bowl-2017/data) of the competition. The images are in [DICOM](https://en.wikipedia.org/wiki/DICOM) format and consist of a group of horizontal slices of the thorax for each patient. This is a 3D reconstruction of a sample lung with the competition data. It was computed with this [script](https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial).  

<p align="center">
<img src="https://migonzastorage.blob.core.windows.net/projects/data_science_bowl_2017/lung_bright.png" alt="sample" width="40%"/>
</p>

The biggest file `stage1.7z` occupies 67Gb. When the file is uncompressed it occupies 141Gb. In case you need more space, you ca easily attach an external drive to your Azure VM. Here you can find the instructions for [Linux](https://docs.microsoft.com/en-us/azure/virtual-machines/virtual-machines-linux-attach-disk-portal?toc=%2fazure%2fvirtual-machines%2flinux%2ftoc.json). An alternative route is to attach a [fileshare disk](https://docs.microsoft.com/en-us/azure/storage/storage-how-to-use-files-linux). With [this script](https://github.com/miguelgfierro/scripts/blob/master/mount_azure_fileshare.sh) the fileshare can be attached in seconds in an Azure DSMV.


In [ ]:
#Load libraries
import sys,os
import numpy as np
import dicom
import glob
from sklearn import cross_validation
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import time
from cntk import load_model
from cntk.ops import combine
import pkg_resources
from lightgbm.sklearn import LGBMRegressor
from scipy.stats import gmean


print("System version: {}".format(sys.version))
print("CNTK version: {}".format(pkg_resources.get_distribution("cntk").version))

## Function definition

The first step is to set the path and variables.

In [ ]:
#Put here the number of your experiment
EXPERIMENT_NUMBER = '0042' 

#Put here the path to the downloaded ResNet model
#https://migonzastorage.blob.core.windows.net/deep-learning/models/cntk/imagenet/ResNet_152.model
MODEL_PATH='/datadrive/pretrained_models/ResNet_152.model' 

#Maximum batch size for the network to evaluate. 
BATCH_SIZE=60

#Number of splits in K-fold cross validation
N_SPLITS=5 

#Put here the path where you downloaded all kaggle data
DATA_PATH='/datadrive/lung_cancer'

# Path and variables
STAGE1_LABELS = os.path.join(DATA_PATH, 'stage1_labels.csv')
STAGE1_SAMPLE_SUBMISSION = os.path.join(DATA_PATH, 'stage1_sample_submission.csv')
STAGE1_FOLDER = os.path.join(DATA_PATH, 'stage1')
EXPERIMENT_FOLDER = 'features%s' % EXPERIMENT_NUMBER
FEATURE_FOLDER = os.path.join(DATA_PATH, 'features', EXPERIMENT_FOLDER)
if not os.path.exists(FEATURE_FOLDER): os.makedirs(FEATURE_FOLDER)
SUBMIT_OUTPUT='submit%s.csv' % EXPERIMENT_NUMBER


This is a timer class. 

In [ ]:
# Timer class
class Timer(object):
    def __enter__(self):
        self.start()
        return self
    def __exit__(self, *args):
        self.stop()
    def start(self):
        self.start = time.clock()
    def stop(self):
        self.end = time.clock()
        self.interval = self.end - self.start

This couple of functions collect the images, perform an histogram equalization and are resized to the ImageNet standard size: `224x224`. Also, the images in ImageNet are color images, with three channels, RGB, however, the images from the data science competition are in gray scale. A quick way to adapt the cancer images to the format of ImageNet is to pack them in groups of three. That is what we are doing in the function `get_data_id`. 


In [ ]:
def get_3d_data(path):
    slices = [dicom.read_file(os.path.join(path, s)) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

In [ ]:
def get_data_id(path, plot_data=False):
    sample_image = get_3d_data(path)
    sample_image[sample_image == -2000] = 0
    if plot_data:
        f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        batch.append(tmp)

        if plot_data:
            if cnt < 20:
                plots[cnt // 5, cnt % 5].axis('off')
                tmp = np.array(tmp)
                plots[cnt // 5, cnt % 5].imshow(tmp[0,:,:], cmap='gray')
            cnt += 1

    if plot_data: plt.show()
        
    batch = np.array(batch, dtype='int')
    return batch

In the next figure, we show the complete architecture. Each patient has an arbitrary number of scan images. The images are cropped to `224x244` and packed in groups of 3, to match the format of ImageNet. They are fed to the pretrained network in k batches and then are convoluted in each internal layer, until the penultimate one. This process is preformed using CNTK. The output of the network are the features that we are going to feed to the boosted tree, programmed with LightGBM.  

<p align="center">
<img src="https://migonzastorage.blob.core.windows.net:443/projects/data_science_bowl_2017/resnet_tree_lung2.png" alt="sample" width="60%"/>
</p>


In this sample experiment, we used the last layer of a pretrained ResNet network with 152 layers as featurizer to extract features. CNTK provides other pretrained networks that you can test like [AlexNet](https://www.cntk.ai/Models/AlexNet/AlexNet.model), [AlexNet with Batch Normalization](https://www.cntk.ai/Models/AlexNet/AlexNetBS.model) and [ResNet with 18 layers](https://www.cntk.ai/Models/ResNet/ResNet_18.model).  
 
Please note the name of the last layer of pretrained network is named as `z.x` in CNTK, which should be specified when use for featurizer. 

In [ ]:
def get_extractor():
    node_name = "z.x"
    loaded_model  = load_model(MODEL_PATH)
    node_in_graph = loaded_model.find_by_name(node_name)
    output_nodes  = combine([node_in_graph.owner])
    return output_nodes

We evaluate the images in batches. When using GPU, for a small network like ResNet18, the images of the patients can be evaluated in one step. However, for a big network like ResNet 152, the memory is too high, and it doesn't fit in a K80 GPU, therefore, we split the data into chunks and evaluate them sequentially.

In [ ]:
def batch_evaluation(model, data, batch_size=50):
    num_items = data.shape[0]
    chunks = np.ceil(num_items / batch_size)
    data_chunks = np.array_split(data, chunks, axis=0)
    feat_list = []
    for d in data_chunks:
        feat = model.eval(d)
        feat_list.append(feat)
    feats = np.concatenate(feat_list, axis=0)
    return feats

This function is responsible of computing the features. For that, given a batch of images we just compute the forward propagation in the network. The resulting features are saved as a numpy array.

In [ ]:
def calc_features(verbose=False):
    net = get_extractor()
    for folder in glob.glob(os.path.join(STAGE1_FOLDER, '*')):
        foldername = os.path.basename(folder)
        foldername_npy = foldername + '.npy'
        if os.path.isfile(os.path.join(FEATURE_FOLDER, foldername_npy)):
            if verbose: print("Features in %s already computed" % (os.path.join(FEATURE_FOLDER, foldername)))
            continue
        batch = get_data_id(folder)
        if verbose:
            print("Batch size:")
            print(batch.shape)
        feats = batch_evaluation(net, batch, BATCH_SIZE)
        if verbose:
            print(feats.shape)
            print("Saving features in %s" % os.path.join(FEATURE_FOLDER, foldername))
        np.save(os.path.join(FEATURE_FOLDER, foldername), feats)

We need to perform a feature engineering process. We are going to perform a dimensionality reduction with PCA

In [ ]:
#Get the feature size of the network penultimante layer 
def get_feature_shape(df, verbose=True):
    feat0 = np.load(os.path.join(FEATURE_FOLDER,'%s.npy' % str(df['id'].iloc[0]))).squeeze()
    feat_shape = feat0.shape[1]
    if verbose: print("Size of the penultimate layer (feature size): {}".format(feat_shape))
    return feat_shape

In [ ]:
def feature_engineering(filename, is_train, verbose=True):
    df = pd.read_csv(filename)
    if verbose: print("Training data of size {}".format(df.shape))
    feat_shape = get_feature_shape(df, verbose)
    x = np.zeros((df.shape[0], feat_shape))
    for i, id in enumerate(df['id'].tolist()):
        feat = np.load(os.path.join(FEATURE_FOLDER,'%s.npy' % str(id))).squeeze()
        feat_pca = PCA(n_components=1).fit_transform(feat.transpose())
        x[i,:] = feat_pca.squeeze()
    if is_train:
        y = df['cancer'].as_matrix()
    else:
        y = None
    return x,y

We can create a custom evaluation function for LightGBM. We want to use the same metric that is used by Kaggle to compute the score in the leaderboard.

In [ ]:
#Custom eval function expects a callable with following functions        
def loglikelood(y_true, y_pred):
    eval_result = log_loss(y_true, y_pred)
    eval_name = 'log_loss'
    is_bigger_better = False
    return eval_name, eval_result, is_bigger_better

Once we have the features computed we can feed them in a boosted tree. Each numpy array contains the features of the images corresponding to one patient, and they are labelled as positive (the patient has cancer) or negative (the patient doesn't have cancer). We can average and flatten all the features to obtain a one dimensional vector representing one observation. The metric use in the tree optimization is logloss, there are [other metrics](https://github.com/Microsoft/LightGBM/blob/7426ac3cbd9ae27b5d734a54e5e9880623beea43/docs/Parameters.md#metric-parameters) that can be applied. 

In [ ]:
def train_lightgbm(x, y, verbose=True):
    skf = KFold(n_splits=N_SPLITS, random_state=2048, shuffle=True)
    clfs = []
    if verbose: print("Computing LightGBM boosted tree using {} kfold cross validation".format(N_SPLITS))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index]

        clf = LGBMRegressor(max_depth=6,
                            num_leaves=21,
                            n_estimators=5000,
                            min_child_weight=30,
                            learning_rate=0.01,
                            nthread=24,
                            boosting_type='gbdt',
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=42)

        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric=loglikelood, early_stopping_rounds=300)
        clfs.append(clf)
        
    return clfs

We create a wrapper of the training function. That way we make the code extendable in case another user wants to try other training method.  

In [ ]:
def compute_training(verbose=True):
    with Timer() as t:
        x, y = feature_engineering(STAGE1_LABELS, is_train=True, verbose=verbose)
        clf = train_lightgbm(x, y, verbose)
    if verbose: print("Training took %.03f sec.\n" % t.interval)
    return clf

Once we have trained the model, we can use it to compute the results in the validation set. Again, this function is easily extendable in case a person want to use another model (always the scikit-learn interface `predict` has to be maintained).

In [ ]:
def compute_prediction(clfs, verbose=True):
    preds = []
    with Timer() as t:
        x, _ = feature_engineering(STAGE1_SAMPLE_SUBMISSION, is_train=False, verbose=verbose)
        for clf in clfs:
            preds.append(np.clip(clf.predict(x),0.0001,1))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval)        
    df['cancer'] = pred
    return df

We finally save the results as a csv file.

In [ ]:
def save_results(df, verbose=True):
    df.to_csv(SUBMIT_OUTPUT, index=False)
    if verbose: print("Results saved in {}".format(SUBMIT_OUTPUT))

## Execution

Plot an example of images with cancer. The plot shows horizontal slices of the thorax. This is the result.
<p align="center">
<img src="https://migonzastorage.blob.core.windows.net/projects/data_science_bowl_2017/output_10_0.png" alt="sample" width="50%"/>
</p>

In [ ]:
path_cancer = os.path.join(STAGE1_FOLDER, 'fe45462987bacc32dbc7126119999392')
data_batch = get_data_id(path_cancer, False)
print("Data batch size: ", data_batch.shape)

Plot example of images without cancer. This is the result:
<p align="center">
<img src="https://migonzastorage.blob.core.windows.net/projects/data_science_bowl_2017/output_12_0.png" alt="sample" width="50%"/>
</p>

In [ ]:
path_no_cancer = os.path.join(STAGE1_FOLDER, 'ffe02fe7d2223743f7fb455dfaff3842')
data_batch = get_data_id(path_no_cancer, False)
print("Data batch size: ", data_batch.shape)

Get CNN.

In [ ]:
%%time
feat = get_extractor()
print(feat)

Compute features. Using a K80 GPU in an Azure NC24 the computation time was: `53min 7s`.

In [ ]:
%%time
# Calculate features
calc_features(verbose=False)

Train the boosted tree. We train the tree with early stoping of 300 rounds, which means that the optimization will stop if the validation score doesn't improve for 300 rounds. You can change the parameters of the model, with `learning_rate=0.001` it took 4.2min in a NC24 machine, these are the results:

```bash
[1]	valid_0's l2: 0.510462
[2]	valid_0's l2: 0.510327
[3]	valid_0's l2: 0.510219
...
...
...
[2483]	valid_0's l2: 0.437271
[2484]	valid_0's l2: 0.437277
[2485]	valid_0's l2: 0.437267
Early stopping, best iteration is:
[2185]	valid_0's l2: 0.436964
Training took 252.097 sec.
```

In [ ]:
%%time
clfs = compute_training(verbose=True)

Compute prediction

In [ ]:
df = compute_prediction(clfs)
print("Results:")
df.head()

Save results to csv. 

In [ ]:
save_results(df)

## Alternative routes and improvements
This notebook is just an example of what you can do with CNTK and LightGBM. Here we present some alternative routes that you can implement based on this script:

- Use other pretrained CNNs: As previously mentioned you can try other networks like [AlexNet](https://www.cntk.ai/Models/AlexNet/AlexNet.model), [AlexNet with Batch Normalization](https://www.cntk.ai/Models/AlexNet/AlexNetBS.model) and [ResNet with 18 layers](https://www.cntk.ai/Models/ResNet/ResNet_18.model). 
- Use transfer learning using a closer domain: The CNN we use is trained in ImageNet, which are natural images. There is another dataset called [LUNA](https://luna16.grand-challenge.org/) that you can use to train a network and then perform transfer learning. A trick that might speed up the convergence is to initialize the weights with a pretrained model (like we are doing in the notebook). 
- Perform image augmentation: You can try to increment the training set by performing transformations in the images. For that you can apply different filters or rotate them.
- Use a customize network with 3D images: CNTK allows [3D convolutions](https://github.com/Microsoft/CNTK/wiki/Convolution). On GPU, 1D, 2D and 3D convolutions will use cuDNN (fast), all other convolutions will use reference engine (slow). You can create a CNN that accepts 3D images. 
- Try tunning the parameters of the boosted tree: You can try to tune the parameters of the tree. For that LightGBM implements the sklearn function `GridSearchCV`. Here you have an [example](https://github.com/Microsoft/LightGBM/blob/b0f7aa508a373b16adbda6cbe66b188a014964d8/examples/python-guide/sklearn_example.py).
- Try some feature engineering: Before training the tree, the features are transformed. You can try to feed the tree without this operation or try a different one.

After the competition was finished, some of the top performers published their solutions.

- **Second place solution**: Julian de Wit was able to get to the second place in the Kaggle competition. Here you can find his [blog](http://juliandewit.github.io/kaggle-ndsb2017/) and here his [code](https://github.com/juliandewit/kaggle_ndsb2017). The solution is based on nodule detectors with a 3D convolutional neural network architecture.
- **Ninth place solution**: A team composed by Andreas Verleysen, Elias Vansteenkiste, Fréderic Godin, Ira Korshunova, Jonas Degrave, Lionel Pigou and Matthias Freiberger, all PhD students and postdocs at Ghent University, were able to reach the ninth position. They published a [blog](https://eliasvansteenkiste.github.io/machine%20learning/lung-cancer-pred/) and the [code](https://github.com/EliasVansteenkiste/dsb3). The pipeline has a 3D CNN for nodule segmentation, one CNN for false positive reduction, another CNN for identifying if the nodule is malignant or not, then transfer learning and finally ensembling.


### Acknowledgements

This notebook is based on this [script published in kaggle](https://www.kaggle.com/hoaphumanoid/data-science-bowl-2017/cntk-and-lightgbm-quick-start). It was also based in this [other script](https://www.kaggle.com/drn01z3/data-science-bowl-2017/mxnet-xgboost-baseline-lb-0-57). 